# 📚 Multilingual Chat with PDF (Powered by Sutra)

### 📌 1. Install Required Packages

In [ ]:
!pip install -q langchain langchain_openai langchain-community faiss-cpu requests pypdf python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.6 MB/s eta 0:00:00


### 📌 STEP 2 : Setup API Keys

In [ ]:
import os
from google.colab import userdata

# Set the API key from Colab secrets
os.environ["SUTRA_API_KEY"] = userdata.get("SUTRA_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

###📌 STEP 3 :  Load the PDF Document

In [ ]:
# 📍 STEP 3: Load the PDF Document (replace 'example.pdf' as needed)
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/GENAI.pdf")
documents = loader.load()
print(f"Loaded {len(documents)} pages.")


Loaded 3 pages.


###📌 STEP 4 :  Split Documents into Chunks

In [ ]:
# 📍 STEP 4: Split Documents into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)
print(f"Split into {len(chunks)} chunks.")


Split into 6 chunks.


###📌 STEP 5 :  Create Embeddings + FAISS Vector Store

In [ ]:
# 📍 STEP 5: Create Embeddings + FAISS Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()


###📌 STEP 6 :  Set Up Conversation Memory and RAG Chain

In [ ]:
# 📍 STEP 6: Set Up Conversation Memory and RAG Chain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# RAG model using Sutra
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(
        api_key=os.getenv("SUTRA_API_KEY"),
        base_url="https://api.two.ai/v2",
        model="sutra-v2",
        temperature=0.5
    ),
    retriever=retriever,
    memory=memory
)


###📌 STEP 7 :  Ask Questions (Supports Multiple Languages)

In [ ]:
# 📍 STEP 7: Ask Questions (Supports Multiple Languages)
def ask_question(question, language="English"):
    rag_response = rag_chain.invoke(question)
    context = rag_response["answer"]

    prompt = f"""
    You are a helpful assistant that answers questions about documents.
    Use the following context to answer the question:

    CONTEXT:
    {context}

    Please respond in {language}.

    Question: {question}
    """

    chat = ChatOpenAI(
        api_key=os.getenv("SUTRA_API_KEY"),
        base_url="https://api.two.ai/v2",
        model="sutra-v2",
        temperature=0.7
    )

    from langchain.schema import HumanMessage
    response = chat.invoke([HumanMessage(content=prompt)])
    return response.content


###📌 STEP 8: Try It Out!

In [ ]:
# 📍 STEP 8: Try It Out!
response = ask_question("list all skills", language="Telugu")
print("🔹 Response:\n", response)


🔹 Response:
 **స్కిల్స్:**

1. **ప్రోగ్రామింగ్ భాషలు:** Python
2. **ఫ్రేమ్‌వర్క్‌లు & టూల్స్:** TensorFlow, PyTorch, LangChain, LlamaIndex, Streamlit, Flask
3. **జనరేటివ్ AI సాంకేతికతలు:** Open-source మరియు paid LLM మోడల్స్ (Llama2, Mistral, OpenAI, Google Gemini Pro)
4. **వెక్టర్ డేటాబేసులు:** ChromaDB, Pinecone
5. **డేటాబేస్ నిర్వహణ:** DataStax Cassandra DB (ఉత్పత్తి వాతావరణాల్లో), MySQL, MongoDB
6. **తెరవెనుక ప్లాట్‌ఫారమ్‌లు:** AWS Bedrock, AWS (EC2, Lambda), Azure Functions, Hugging Face Spaces
7. **AI/ML సాంకేతికతలు:** కస్టమ్ డేటాతో ఫైన్-ట్యూనింగ్, వెక్టర్ ఎంబెడింగ్, NLP, న్యూరల్ నెట్‌వర్క్ ఆప్టిమైజేషన్, MLOPS, Docker, Kubernetes
8. **మృదువైన నైపుణ్యాలు:** విశ్లేషణాత్మక ఆలోచన, సమస్య పరిష్కారం, టీమ్ వర్క్, సమర్థవంతమైన కమ్యూనికేషన్


##✅ Finally Integrated UI

In [ ]:
# 1. Imports
import os
import requests
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import HumanMessage
from tempfile import NamedTemporaryFile

# 2. Setup LLM (Sutra)
def get_sutra_model():
    return ChatOpenAI(
        api_key=os.getenv("SUTRA_API_KEY"),
        base_url="https://api.two.ai/v2",
        model="sutra-v2",
        temperature=0.7
    )

# 3. Load and index PDF
def load_and_index_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(docs)

    embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
    vectorstore = FAISS.from_documents(chunks, embeddings)

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    chain = ConversationalRetrievalChain.from_llm(
        llm=get_sutra_model(),
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return chain

# 4. UI Components

# PDF File Upload widget
pdf_file_upload = widgets.FileUpload(
    accept='.pdf',
    multiple=False,
    description='📁 Upload PDF',
    layout=widgets.Layout(width='300px')
)

load_pdf_button = widgets.Button(
    description="🔄 Load PDF",
    button_style='info',
    layout=widgets.Layout(width='150px')
)

status_output = widgets.Output()

# Language dropdown
languages = [
    "English", "Hindi", "Gujarati", "Bengali", "Tamil",
    "Telugu", "Kannada", "Malayalam", "Punjabi", "Marathi",
    "Urdu", "Assamese", "Odia", "Sanskrit", "Korean",
    "Japanese", "Arabic", "French", "German", "Spanish",
    "Portuguese", "Russian", "Chinese", "Vietnamese", "Thai",
    "Indonesian", "Turkish", "Polish", "Ukrainian", "Dutch",
    "Italian", "Greek", "Hebrew", "Persian", "Swedish",
    "Norwegian", "Danish", "Finnish", "Czech", "Hungarian",
    "Romanian", "Bulgarian", "Croatian", "Serbian", "Slovak",
    "Slovenian", "Estonian", "Latvian", "Lithuanian", "Malay",
    "Tagalog", "Swahili"
]

lang_dropdown = widgets.Dropdown(
    options=languages,
    value="English",
    description='🌐 Language:',
    layout=widgets.Layout(width='300px')
)

chat_output = widgets.HTML(
    value="<div style='padding:10px; font-family:Arial; font-size:14px; height:300px; overflow-y:auto; border:1px solid #ccc; border-radius:5px;'>Chat history will appear here...</div>"
)

user_input = widgets.Text(
    placeholder='Type your message...',
    layout=widgets.Layout(flex='4', width='auto')
)

send_button = widgets.Button(
    description="📤 Send",
    button_style='primary',
    layout=widgets.Layout(flex='1', width='auto')
)

messages = []
conversation_chain = None

# 5. Load PDF Logic
def on_load_pdf(b):
    global conversation_chain
    uploaded_files = pdf_file_upload.value

    with status_output:
        clear_output()
        if not uploaded_files:
            print("❌ Please upload a PDF file first.")
            return
        try:
            print("⏳ Processing uploaded PDF...")

            # Save uploaded content to a temp file
            uploaded_file = list(uploaded_files.values())[0]
            with NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
                tmp.write(uploaded_file['content'])
                tmp_path = tmp.name

            conversation_chain = load_and_index_pdf(tmp_path)
            print("✅ PDF loaded and indexed successfully!")
        except Exception as e:
            print("❌ Error:", e)

load_pdf_button.on_click(on_load_pdf)

# 6. Chat Interaction Logic
def on_send_click(b):
    global conversation_chain
    if conversation_chain is None:
        with status_output:
            clear_output()
            print("❌ Load a PDF first.")
        return

    user_text = user_input.value.strip()
    if not user_text:
        return

    lang = lang_dropdown.value
    messages.append(f"<b style='color:#13f22d;'>You:</b> {user_text}")

    context_response = conversation_chain.invoke(user_text)
    rag_context = context_response['answer']

    system_msg = f"""
You are a helpful assistant answering based on a document.
Use this context: {rag_context}
Always reply in: {lang}
Question: {user_text}
"""

    chat_model = get_sutra_model()
    sutra_response = chat_model.invoke([HumanMessage(content=system_msg)])
    assistant_reply = sutra_response.content.strip()

    messages.append(f"<b style='color:#007acc;'>Assistant ({lang}):</b> {assistant_reply}")

    chat_html = "<br>".join(messages)
    chat_output.value = f"<div style='padding:10px; font-family:Arial; font-size:14px; height:300px; overflow-y:auto; border:1px solid #ccc; border-radius:5px;'>{chat_html}</div>"

    user_input.value = ""

send_button.on_click(on_send_click)

# 7. Final Layout
input_row = widgets.HBox([user_input, send_button])
pdf_row = widgets.HBox([pdf_file_upload, load_pdf_button])

ui = widgets.VBox([
    widgets.HTML("<h3 style='font-family:Arial;'>📚 Multilingual Chat with PDF (Upload from Local)</h3>"),
    pdf_row,
    lang_dropdown,
    chat_output,
    input_row,
    status_output
])

# 8. Display the App
display(ui)
